# Initial implementation

Following up on the previous notebook, we'll start with the implementation of the Hanoi class:

In [ ]:
from typing import Dict, List

class Hanoi:
    def __init__(self, num_discs: int):
        self.num_discs = num_discs
        self._source = []
        self._temporary = []
        self._destination = []
        
        for i in range(num_discs, 0, -1):
            self._source.append(i)
    
    def draw(self):
        towers: Dict[str, List[int]] = {
            "SOURCE": self._source,
            "TEMPORARY": self._temporary,
            "DESTINATION": self._destination,
        }
        empty_level: str = "|"
        tower_base: str = "="

        for name, tower in towers.items():
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(empty_level)
            for disc in range(1, self.num_discs + 1):
                if disc in tower:
                    print("X" * disc)
                else:
                    print("|")
            print(tower_base * self.num_discs + "\n")
    
    def _hanoi(self, source: List[int], destination: List[int], temporary: List[int], num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.append(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    
    def solve(self):
        self._hanoi(
            source=self._source,
            destination=self._destination,
            temporary=self._temporary,
            num_discs=self.num_discs,
        )
    
    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()

In [ ]:
hanoi = Hanoi(num_discs=3)
hanoi.draw_and_solve()

# S: Single Responsibility Principle

By quickly inspecting the class, we can see that it's doing at least the following things:

- Creating and populating the towers
- Displaying the towers
- Solving the problem

In short, we're not following the single responsibility principle.

## Exercise - Plan your changes

What changes would you do in order to follow the SRP?

As with most problems in Software Engineering, there's not a single answer to this. There's always a tradeoff between how strictly you want to apply the principle and other factors like how much work it will take you to apply it, how easy will it be for new people to understand the code, etc.

We could start by making the following changes:
- Create a Tower class
- Create a HanoiTowers class, that will manage the source/destination/temporary towers
- Create a class to display the towers
- Create a class to solve the problem
- The Hanoi class should be responsible only for "coordinating" the previous.

One could argue that we need even more classes, for instance a ```Disc``` class could make sense.

## Solution - Creating a Tower class

We will create a tower class that will take care of the initialization, and make the necessary changes in our Hanoi implementation.

In [ ]:
from typing import List

class Tower:
    def __init__(self):
        """Creates a tower"""
        self._discs: List[int] = []

    def push(self, disc: int) -> None:
        self._discs.append(disc)

    def pop(self) -> int:
        return self._discs.pop()

    def __repr__(self) -> str:
        return repr(self._discs)
    
    def __contains__(self, disc: int) -> bool:
        return disc in self._discs

And here's the new version of the Hanoi class:

In [ ]:
from typing import Dict, List

class Hanoi:
    def __init__(self, num_discs: int):
        self.num_discs = num_discs
        self._source = Tower()
        self._temporary = Tower()
        self._destination = Tower()
        
        for disc in range(num_discs, 0, -1):
            self._source.push(disc)
    
    def draw(self):
        towers: Dict[str, List[int]] = {
            "SOURCE": self._source,
            "TEMPORARY": self._temporary,
            "DESTINATION": self._destination,
        }
        empty_level: str = "|"
        tower_base: str = "="

        for name, tower in towers.items():
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(empty_level)
            for disc in range(1, self.num_discs + 1):
                if disc in tower:
                    print("X" * disc)
                else:
                    print("|")
            print(tower_base * self.num_discs + "\n")
    
    def _hanoi(self, source: Tower, destination: Tower, temporary: Tower, num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    
    def solve(self):
        self._hanoi(
            source=self._source,
            destination=self._destination,
            temporary=self._temporary,
            num_discs=self.num_discs,
        )
    
    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()

Most importantly: let's make sure that the new version still works!

In [ ]:
hanoi = Hanoi(num_discs=3)
hanoi.draw_and_solve()

## Solution - Create the HanoiTowers class

Now we'll create another class that will model the three towers in our problem and take care of the initialization. With this, we're taking that responsibility out of the ```Hanoi``` class.

In [ ]:
from typing import Tuple, Iterator

class HanoiTowers:
    SOURCE: str = "SOURCE"
    TEMP: str = "TEMPORARY"
    DEST: str = "DESTINATION"
    def __init__(self, num_discs: int):
        self.source: Tower = Tower()
        self.destination: Tower = Tower()
        self.temporary: Tower = Tower()
        self.num_discs = num_discs
        
        for disc in range(num_discs, 0, -1):
            self.source.push(disc)

    def __iter__(self) -> Iterator[Tuple[str, Tower]]:
        yield from {
            self.SOURCE: self.source,
            self.TEMP: self.temporary,
            self.DEST: self.destination,
        }.items()
    
    def __len__(self) -> int:
        return 3

Again, let's adapt the ```Hanoi``` class and make sure that it works

In [ ]:
from typing import List

class Hanoi:
    def __init__(self, towers: HanoiTowers):
        self.towers = towers
    
    def draw(self):
        empty_level: str = "|"
        tower_base: str = "="

        for name, tower in self.towers:
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(empty_level)
            for disc in range(1, self.towers.num_discs + 1):
                if disc in tower:
                    print("X" * disc)
                else:
                    print("|")
            print(tower_base * self.towers.num_discs + "\n")
    
    def _hanoi(self, source: Tower, destination: Tower, temporary: Tower, num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    
    def solve(self):
        self._hanoi(
            source=self.towers.source,
            destination=self.towers.destination,
            temporary=self.towers.temporary,
            num_discs=self.towers.num_discs,
        )
    
    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()

As always, let's test if our classes are working

In [ ]:
num_discs: int = 3
towers = HanoiTowers(num_discs=num_discs)
hanoi = Hanoi(towers=towers)
hanoi.draw_and_solve()

## Solution - Create a class to display the towers

Next, we'll create a class that will be responsible for displaying the towers (i.e. the ```draw``` method in our initial implementation).

In [ ]:
from typing import List

class HanoiPrinter:
    def draw(self, towers: HanoiTowers) -> None:
        empty_level: str = "|"
        tower_base: str = "="

        for name, tower in towers:
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(empty_level)
            for disc in range(1, towers.num_discs + 1):
                if disc in tower:
                    print("X" * disc)
                else:
                    print("|")
            print(tower_base * towers.num_discs + "\n")

And let's adapt the ```Hanoi``` class again. This time we have moved a whole method to another class, so we'll *inject* the printer to the ```Hanoi``` class. Note that we have also created a new property in the class, ```towers``` that will allow us to get the three towers very easily.

In [ ]:
from typing import List

class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter):
        self.towers = towers
        self.printer = printer

    def draw(self):
        self.printer.draw(towers=self.towers)
    
    def _hanoi(self, source: Tower, destination: Tower, temporary: Tower, num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    
    def solve(self):
        self._hanoi(
            source=self.towers.source,
            destination=self.towers.destination,
            temporary=self.towers.temporary,
            num_discs=self.towers.num_discs,
        )
    
    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()

Again, let's test that it still works.

In [ ]:
num_discs: int = 3
towers = HanoiTowers(num_discs=num_discs)
printer = HanoiPrinter()
hanoi = Hanoi(towers=towers, printer=printer)
hanoi.draw_and_solve()

### Extra practice - refactor

Refactor the ```HanoiPrinter``` class so that we can customize the characters we use for empty levels ("|"), for the discs ("X") and for the tower base ("=").


Currently, all of these characters (and the number of spaces) are hard-coded in the ```draw``` method. Instead of that, we could just have them as parameters in our class constructor, with default values, something like this:

In [ ]:
from typing import List
from dataclasses import dataclass

@dataclass
class HanoiPrinter:
    empty_level: str = "|"
    tower_base_fill: str = "="
    disc_fill: str = "X"
    
    def draw(self, towers: List[Tower]) -> None:
        for name, tower in towers:
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(self.empty_level)
            for disc in range(1, towers.num_discs + 1):
                if disc in tower:
                    print(self.disc_fill * disc)
                else:
                    print(self.empty_level)
            print(self.tower_base_fill * towers.num_discs + "\n")


num_discs: int = 3
towers = HanoiTowers(num_discs=num_discs)
printer = HanoiPrinter(disc_fill="O", tower_base_fill="**")
hanoi = Hanoi(towers=towers, printer=printer)
hanoi.draw_and_solve()

### Extra practice - Default printer

People using your library are now complaining that they need to also create a ```HanoiPrinter``` instance, but they are just happy with how it was printed before, and will never want to change it. How would you fix this problem?

There are several ways to tackle this issue, for instance:

- Make the ```printer``` argument optional, and create a ```HanoiPrinter``` if none is provided. This is one of the simplest solutions, but adds direct coupling between the two classes: now the Hanoi class now needs to know not only how to "use" a HanoiPrinter, but also how to create it. Nonetheless, let's show how this would look:

```python
from typing import Optional

class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: Optional[HanoiPrinter] = None):
        # The HanoiPrinter() will only be called when printer is None.
        self.printer = printer or HanoiPrinter()
        # ...
```

- Keep the constructor as is, and add a new static method that will create the printer for us. This is quite similar to the previous method, with the advantage that you can be sure that the old constructor still works, since it wasn't changed. In short, you're just *adding* code rather than *changing* it:

```python
class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter):
        # This stays the same as before...
    
    @staticmethod
    def with_default_printer(self, towers: HanoiTowers) -> "Hanoi":
        printer = HanoiPrinter()
        return Hanoi(towers=towers, printer=printer)
```

Note that the last option still has the issue with coupling: it needs to know how to create a HanoiPrinter class, even if we have made sure that there are no changes in the old way of constructing the ```Hanoi``` class.

The same idea could be applied to the creation of the ```HanoiTowers``` class, but as you see, the number of cases to handle starts to multiply.

## Solution - Create a class to solve the problem

The next "responbility" we'll be taking out of the ```Hanoi``` class is solving the problem. We will move that to a separate class that will take care of that:

In [ ]:
class HanoiSolver:
    def _hanoi(self, source: Tower, destination: Tower, temporary: Tower, num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    def solve(self, towers: HanoiTowers):
        self._hanoi(
            source=towers.source,
            destination=towers.destination,
            temporary=towers.temporary,
            num_discs=towers.num_discs,
        )

And now take these changes into the ```Hanoi``` class:

In [ ]:
class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter, solver: HanoiSolver):
        self.towers = towers
        self.printer = printer
        self.solver = solver

    def draw(self):
        self.printer.draw(towers=self.towers)
    
    def solve(self):
        self.solver.solve(towers=self.towers)

    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()

Last but not least, the test:

In [ ]:
num_discs: int = 3
towers = HanoiTowers(num_discs=num_discs)
printer = HanoiPrinter()
solver = HanoiSolver()
hanoi = Hanoi(towers=towers, printer=printer, solver=solver)
hanoi.draw_and_solve()

## Solution - The Hanoi class should be responsible only for "coordinating"

We actually got this last point "for free" with the previous ones. Now the ```Hanoi``` class is doing nothing but coordinating the different parts, and providing the convenience method ```draw_and_solve```.

As discussed before, we could also have a static method that creates all the default components to make it even easier to get started:

In [ ]:
class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter, solver: HanoiSolver):
        self.towers = towers
        self.printer = printer
        self.solver = solver

    def draw(self):
        self.printer.draw(towers=self.towers)
    
    def solve(self):
        self.solver.solve(towers=self.towers)

    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()
    
    @staticmethod
    def create(num_discs: int) -> "Hanoi":
        towers = HanoiTowers(num_discs=num_discs)
        printer = HanoiPrinter()
        solver = HanoiSolver()
        
        return Hanoi(towers=towers, printer=printer, solver=solver)

So now we can do this:

In [ ]:
hanoi = Hanoi.create(num_discs=3)
hanoi.draw_and_solve()

## Extra exercises

Here are some ideas, in case you want to practice further:

- Create a ```Disc``` class
- Create a HanoiPrinter class that can "draw" to a file, plot the problem as a matrix, or any other thing you can think of

# Final Code

This is the code for all the classes, together:

In [ ]:
# Tower class
from typing import List

class Tower:
    def __init__(self) -> None:
        """Creates a tower"""
        self._discs: List[int] = []        

    def push(self, disc: int) -> None:
        self._discs.append(disc)

    def pop(self) -> int:
        return self._discs.pop()

    def __repr__(self) -> str:
        return repr(self._discs)
    
    def __contains__(self, disc: int) -> bool:
        return disc in self._discs

# HanoiTowers class
from typing import Tuple, Iterator

class HanoiTowers:
    SOURCE: str = "SOURCE"
    TEMP: str = "TEMPORARY"
    DEST: str = "DESTINATION"
    def __init__(self, num_discs: int):
        self.source: Tower = Tower()
        self.destination: Tower = Tower()
        self.temporary: Tower = Tower()
        self.num_discs = num_discs
        
        for disc in range(num_discs, 0, -1):
            self.source.push(disc)

    def __iter__(self) -> Iterator[Tuple[str, Tower]]:
        yield from {
            self.SOURCE: self.source,
            self.TEMP: self.temporary,
            self.DEST: self.destination,
        }.items()
    
    def __len__(self) -> int:
        return 3

# HanoiPrinter class
from typing import List
from dataclasses import dataclass

@dataclass
class HanoiPrinter:
    empty_level: str = "|"
    tower_base_fill: str = "="
    disc_fill: str = "X"
    
    def draw(self, towers: HanoiTowers) -> None:
        for name, tower in towers:
            title = f"{name} Tower"
            print(title)
            print("=" * len(title) + "\n")

            print(self.empty_level)
            for disc in range(1, towers.num_discs + 1):
                if disc in tower:
                    print(self.disc_fill * disc)
                else:
                    print(self.empty_level)
            print(self.tower_base_fill * towers.num_discs + "\n")

# HanoiSolver
class HanoiSolver:
    def _hanoi(self, source: Tower, destination: Tower, temporary: Tower, num_discs: int):
        if num_discs == 1:
            # Just move disc from source to destination
            destination.push(source.pop())
        else:
            # Move N - 1 discs from the source to the temporary tower.
            # This means we "swap" destination and temporary
            self._hanoi(
                source=source,
                destination=temporary,
                temporary=destination,
                num_discs=num_discs - 1,
            )
            # Move the remaining disc from the source to the destination
            self._hanoi(
                source=source,
                destination=destination,
                temporary=temporary,
                num_discs=1,
            )
            # Finally, move the N - 1 discs from the first step to the destination.
            # This means that we are swapping the source and the temporary towers.
            self._hanoi(
                source=temporary,
                destination=destination,
                temporary=source,
                num_discs=num_discs - 1,
            )
    def solve(self, towers: HanoiTowers):
        self._hanoi(
            source=towers.source,
            destination=towers.destination,
            temporary=towers.temporary,
            num_discs=towers.num_discs,
        )

# Hanoi class
class Hanoi:
    def __init__(self, towers: HanoiTowers, printer: HanoiPrinter, solver: HanoiSolver):
        self.towers = towers
        self.printer = printer
        self.solver = solver

    def draw(self):
        self.printer.draw(towers=self.towers)
    
    def solve(self):
        self.solver.solve(towers=self.towers)

    def draw_and_solve(self):
        print("Initial State:\n")
        self.draw()
        self.solve()
        print("\n\nFinal State:\n")
        self.draw()
    
    @staticmethod
    def create(num_discs: int) -> "Hanoi":
        towers = HanoiTowers(num_discs=num_discs)
        printer = HanoiPrinter()
        solver = HanoiSolver()
        
        return Hanoi(towers=towers, printer=printer, solver=solver)